In [1]:
#%% library import
import numpy as np
import pandas as pd
import torch as tc
import torch
import pprint
import pickle

from torch.autograd import Variable
from sklearn.utils import shuffle
from sklearn.preprocessing import Normalizer
from bayes_opt import BayesianOptimization
from matplotlib import pyplot as plt
from matplotlib import image as img
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from functools import partial

In [ ]:
#%% mini_dataset loading
with open("datasets/KIBA/trainset.txt", "rb") as fp:
    trainset = pickle.load(fp)

with open("datasets/KIBA/validset.txt", "rb") as fp:
    validset = pickle.load(fp)

In [ ]:
#%% Make collate func.
def collate(samples):
    # The input `samples` is a list of pairs [(graph, label),(graph, label)].
    descs, images, labels = map(list, zip(*samples))
    return tc.tensor(descs, dtype=tc.float).cuda(), tc.tensor(images, dtype=tc.float).cuda(), tc.tensor(labels).cuda()

In [2]:
#%% learning module 선언
class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()    # method 상속받고 __init__()은 여기서 하겠다.
        
        self.dslayers = nn.Sequential(     
                        nn.Linear(9577, 2048),
                        nn.BatchNorm1d(2048),
                        nn.ReLU(),
                        nn.Dropout(0.5),
                        nn.Linear(2048, 512),
                        nn.BatchNorm1d(512),
                        nn.ReLU(),
                        nn.Dropout(0.2),
                        nn.Linear(512, 128),
                        )
    
        self.imlayers = nn.Sequential(
                        BasicConv2d(1, 4, kernel_size=4, stride=1),
                        InceptionResnet_Ablock(scale=0.17),
                        InceptionResnet_Ablock(scale=0.17),
                        InceptionResnet_Ablock(scale=0.17),
                        Reduction_Ablock(), 
                        InceptionResnet_Bblock(scale=0.10),
                        InceptionResnet_Bblock(scale=0.10),
                        InceptionResnet_Bblock(scale=0.10),
                        Reduction_Bblock(),
                        InceptionResnet_Cblock(scale=0.20),
                        InceptionResnet_Cblock(scale=0.20),
                        InceptionResnet_Cblock(scale=0.20)
                        )
        
        self.avgpool = nn.AvgPool2d(68, count_include_pad=False)
        self.regress = nn.Linear(208, 1)

    def forward(self, desc, image):
        ds_h = self.dslayers(desc)
        dim = 1
        for d in ds_h.size()[1:]:
            dim = dim * d
        ds_o = ds_h.view(-1, dim)
        
        im_h = self.imlayers(image)
        im_h = self.avgpool(im_h)
        dim = 1
        for d in im_h.size()[1:]: #16, 4, 4
            dim = dim * d
        im_o = im_h.view(-1, dim)
        
        cat = tc.cat((ds_o, im_o), 1)
       
        return self.regress(cat)

In [ ]:
#%% Set hyperparameter
hp_d = {}

# FIXME: 학습 관련 하이퍼파라미터
hp_d['batch_size'] = 32
hp_d['num_epochs'] = 1000

hp_d['init_learning_rate'] = 10 ** -3.70183
hp_d['eps'] = 10 ** -8.39981
hp_d['weight_decay'] = 10 ** -3.59967

In [ ]:
#%% Training
tr_data_loader = DataLoader(trainset, batch_size=hp_d['batch_size'], shuffle=False, collate_fn=collate)
#va_data_loader = DataLoader(validset, batch_size=hp_d['batch_size'], shuffle=False, collate_fn=collate)

model = Regressor().to(torch.device('cuda:0'))
# model = Regressor().to(torch.device('cpu'))

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
loss_func = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=hp_d['init_learning_rate'], 
    weight_decay=hp_d['weight_decay'], eps=hp_d['eps'])
print('total params:', total_params)

tr_epoch_losses = []
va_epoch_losses = []

for epoch in range(hp_d['num_epochs']):                           #!! epoch-loop
    # training session
    model.train()
    tr_epoch_loss = 0

    for iter, (desc, image, label) in enumerate(tr_data_loader):  #!! batch-loop
        prediction = model(desc, image).view(-1)
        loss = loss_func(prediction, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        tr_epoch_loss += loss.detach().item()
        
        if np.mod(iter, 20) == 0:
            print((iter*hp_d['batch_size']), '/', len(trainset))
            

    tr_epoch_loss /= (iter + 1)
    print('Training epoch {}, loss {:.4f}'.format(epoch, tr_epoch_loss))
    tr_epoch_losses.append(tr_epoch_loss)

# ===========================================================================

#%% ===========================================================================
    # validation session
#     model.eval()
#     va_epoch_loss = 0

#     for iter, (desc, image, label) in enumerate(va_data_loader):  # batch-loop
#         prediction = model(desc, image).view(-1).cuda()
#         loss = loss_func(prediction, label).cuda()
        
#         va_epoch_loss += loss.detach().item()
        
#         va_pre_global = prediction         # 전역변수 할당
#         va_lab_global = label              # 전역변수 할당
        
#     va_epoch_loss /= (iter + 1)
#     print('Validation epoch {}, loss {:.4f}'.format(epoch, va_epoch_loss))
#     va_epoch_losses.append(va_epoch_loss)

# print('min_val_loss:', min(va_epoch_losses))


#%%
#bayes_optimizer = BayesianOptimization(
#    f=train_and_validate,
#    pbounds={
#        'init_learning_rate_log': (-4.5,-1.5),
#        'weight_decay_log': (-4.5,-1.5),
#        'eps': (-9, -7)},
#    random_state=0,
#    verbose=2)
#
#bayes_optimizer.maximize(init_points=10, n_iter=200, acq='ei', xi=0.01)    # FIXME, 제너레이터 생성
#for i, res in enumerate(bayes_optimizer.res):
#    print('Iteration {}: \n\t{}'.format(i, res))
#    
#print('Final result: ', bayes_optimizer.max)
#
#
#for i in [ 95, 104, 157]:
#    print(bayes_optimizer.res[i])
#
#
##%%
#prepre = np.array(tr_pre_global.detach().cpu())
#lablab = np.array(tr_lab_global.detach().cpu())
#
#plt.plot(np.arange(512), lablab[:512], prepre[:512])
#corr = np.corrcoef(lablab, prepre)[1,0]
#
#print(np.var(lablab))
#print(np.var(prepre))